In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

## Read in assigned reads from Arianna's coverm run, allowing `coverm` to collapse dups

In [2]:
output_folder="/vortexfs1/omics/alexander/halexander/2020-tara-mag-abund/output_clustered_prok_euk"
initial_df = pd.read_csv(os.path.join(output_folder,'ERR1726699.coverm.abundance.tab'), 
                         sep='\t', index_col=0)
lengths = initial_df["ERR1726699_1.trimmed.fastq.gz Length"].drop('unmapped')
initial_df.columns = initial_df.columns.str.split(' ').str[1:].str.join('_')
all_reads_new = pd.DataFrame(index = initial_df.index)
for f in glob.glob(os.path.join(output_folder,'*tab')):
    name = os.path.basename(f).split('.')[0]
    if name == 'ERR1726706':
        pass
    else:
        tt = pd.read_csv(f, sep='\t', index_col=0)
        tt.columns = tt.columns.str.split(' ').str[1:].str.join('_')
        all_reads_new[name+"_Reads"] = tt['Read_Count']
        all_reads_new[name] = tt['RPKM']
all_reads_new= all_reads_new.drop('unmapped')

In [3]:
renamed = pd.concat([pd.read_csv('/vortexfs1/omics/alexander/halexander/2021-tara-final-paper/rename/renamed-eukaryotic-mags.tsv', sep='\t'),
                     pd.read_csv('/vortexfs1/omics/alexander/halexander/2021-tara-final-paper/rename/renamed-prokaryotic-mags.tsv', sep='\t')])
new_take=pd.wide_to_long(all_reads_new.loc[:,[curr for curr in all_reads_new.columns \
                                          if "Reads" not in curr]].reset_index(),
                         stubnames="ERR",i="Genome",j="SampNum").reset_index().\
    rename({"ERR":"NewTake"},axis="columns")
all_reads_reads=all_reads_new.loc[:,[curr for curr in all_reads_new.columns \
                                          if "Reads" in curr]].reset_index()
all_reads_reads.columns=[curr.split("_Reads")[0] for curr in all_reads_reads.columns]
new_take_reads=pd.wide_to_long(all_reads_reads,
                         stubnames="ERR",i="Genome",j="SampNum").reset_index().\
    rename({"ERR":"NewTake"},axis="columns")

## Get overall reads for normalization

In [5]:
read_counts = pd.DataFrame(columns = ['all_reads', 'trimmed_surviving'])
with open('/vortexfs1/omics/alexander/halexander/2020-tara-mag-abund/trimmed-read-stats', 'r') as f:
    for line in f:
        l=line.split(':')
        name = l[0].split('.')[0]
        all_reads = int(l[2].strip().split(' ')[0])
        trimmed_reads_surving_pairs = int(l[3].split(' ')[1])
        read_counts.loc[name] = [all_reads_new, trimmed_reads_surving_pairs]


In [6]:
library_size = read_counts['trimmed_surviving']

In [7]:
summed_reads = all_reads_new.sum()

X = total reads recruiting to a genome

l = length of genome in Kb

N = total number of trimmed reads in millions

\begin{align}
RPKM_{i} & = \frac{X_i}{l_i  N } * 10^9
\end{align}

## Calculate RPKM from `coverm` run with clustering

In [8]:
lengths = initial_df["Length"].drop('unmapped')
rename_lengths = pd.merge(lengths.reset_index(),renamed,left_on="Genome",right_on="new_mag_name")
lengths_rpkm = pd.Series(list(rename_lengths["Length"]),
                         index=rename_lengths.new_mag_name)
new_take_reads["Sample"] = ["ERR"+str(curr) for curr in new_take_reads.SampNum]
new_take_pivot = new_take_reads.merge(renamed,left_on="Genome",right_on="new_mag_name").\
                    pivot_table(values='NewTake', 
                                                                             index=['old_mag_name',
                                                                                    'new_mag_name'],
                    columns=['Sample']).reset_index()

lengths_rpkm = lengths[list(new_take_pivot["new_mag_name"])]
rpkm_new_take_reads = (new_take_pivot.drop(["old_mag_name","new_mag_name"],axis=1)).\
    div(library_size[new_take_pivot.drop(["old_mag_name","new_mag_name"],axis=1).\
                     columns]).div(lengths_rpkm.reset_index(drop=True), axis=0)*1e9
rpkm_new_take_reads["Genome"] = new_take_pivot.new_mag_name

TPM is normalized to all total RPKM
\begin{align}
TPM_{i} & = \frac{RPKM_i}{\sum RPKM} * 10^6
\end{align}

In [10]:
renamed = pd.read_csv('/vortexfs1/omics/alexander/halexander/2021-tara-final-paper/rename/renamed-eukaryotic-mags.tsv', sep='\t')
rpkm_faceted = pd.wide_to_long(rpkm_new_take_reads.reset_index(),stubnames="ERR",i="Genome",j="SampNum").\
    reset_index().rename({"ERR":"Original"},axis="columns").merge(renamed,left_on="Genome",right_on="old_mag_name")

In [11]:
new_take["Sample"] = ["ERR"+str(curr) for curr in new_take.SampNum]
summed_rpkm = new_take.merge(renamed,left_on="Genome",right_on="new_mag_name").pivot_table(values='NewTake', 
                                                                             index=['old_mag_name',
                                                                                    'new_mag_name'],
                    columns=['Sample']).reset_index()
selected = summed_rpkm.loc[:,["ERR" in curr for curr in summed_rpkm.columns]]
summed_rpkm = selected.sum(axis=0)

In [14]:
final_df=pd.DataFrame()
for colname in selected.columns:
    divided_res = (selected[colname]).div(summed_rpkm[colname])
    final_df[colname]=divided_res

In [15]:
tpm=final_df*1e6

In [16]:
for_names=new_take.merge(renamed,left_on="Genome",right_on="new_mag_name").pivot_table(values='NewTake', 
                                                                             index=['new_mag_name'],
                    columns=['Sample']).index

In [17]:
tpm["Genome"]=for_names

In [13]:
new_take.merge(renamed,left_on="Genome",right_on="new_mag_name").pivot_table(values='NewTake', 
                                                                             index=['old_mag_name',
                                                                                    'new_mag_name'],
                    columns=['Sample']).reset_index().to_csv(os.path.join("../input/new_MAG_rpkm.csv"))
new_take.merge(renamed,left_on="Genome",right_on="new_mag_name").pivot_table(values='NewTake', 
                                                                             index=['old_mag_name',
                                                                                    'new_mag_name'],
                    columns=['Sample']).reset_index().to_csv(os.path.join("../input/new_MAG_tpm.csv"))

In [19]:
tpm.to_csv(os.path.join("..","input","MAG_tpm_new_approach.csv"))

new_take.merge(renamed,left_on="Genome",right_on="new_mag_name").pivot_table(values='NewTake', 
                                                                             index=['old_mag_name',
                                                                                    'new_mag_name'],
                    columns=['Sample']).reset_index().to_csv(os.path.join("..","input",
                                                                          "MAG_rpkm_new_approach.csv"))